# Keras Model

Before you begin using Keras, import the required modules. If you are having problems with this step, make sure you have installed all dependencies found in requirements.txt and have selected the correct Python kernel (in the top menu, "Kernel"->"Change kernel"). If you are using virtualenv, your kernel will not show up unless you follow the instructions found [here](https://stackoverflow.com/questions/37891550/jupyter-notebook-running-kernel-in-different-env).

In [1]:
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.optimizers import SGD

from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

## Data Preprocessing

First, we want to grab the dataset from the CSV file. Load it as a Pandas Dataframe so we can easily work with it in further steps.

In [ ]:
def get_dataset(path):
    dataset = pd.read_csv(path)
    np.random.seed(42)
    dataset = dataset.reindex(np.random.permutation(dataset.index))
    return dataset

dataset = get_dataset('data/dataset.csv')
dataset.head()

Now, we'll use functions to get the data and the labels (i.e. the x and the y). We want to make sure these return NumPy arrays so they can be passed into Keras models.

In [ ]:
def get_data(dataset):
    data = np.array(dataset.as_matrix()[:, 2:-1], dtype=np.float64)
    return data

def get_labels(dataset):
    diagnoses = dataset['diagnosis'].map({'M':1, 'B':0})
    return np.array(diagnoses.as_matrix(), dtype=np.uint8)

data, labels = get_data(dataset), get_labels(dataset)
print(data.shape, labels.shape)

Finally, we'll split the data and labels into train, validation, and test sets.

In [ ]:
def split_data(data, labels, split):
    train_ratio, val_ratio, test_ratio = split
    num_examples = labels.shape[0]
    train_bound, val_bound = int(train_ratio*num_examples), int(train_ratio*num_examples) + int(val_ratio*num_examples)
    
    train = {'data': data[:train_bound], 'labels': labels[:train_bound]}
    val = {'data': data[train_bound:val_bound], 'labels': labels[train_bound:val_bound]}
    test = {'data': data[val_bound:], 'labels': labels[val_bound:]}
    
    return train, val, test
    
train, val, test = split_data(data, labels, (.7, .2, .1))

Next, we want to scale our data so that each feature has mean 0 and variance 1. This is useful because it improves the stability of training our neural network. This makes it possible to train using more sophisticated networks and get better results.

In [ ]:
scaler = StandardScaler()
scaler.fit(train['data'])
train['data'] = scaler.transform(train['data'])
val['data'] = scaler.transform(val['data'])
test['data'] = scaler.transform(test['data'])

## Model Architecture and Training

Let's define our model's architecture now. This is the key part to implement. We want a 4-layer fully-connected network that can be used for binary classification.

In [ ]:
def create_model():
    
     model = Sequential()
     # TODO: Build model!

model = create_model()
model.summary()

Now let's train our model! We want to not only train the model with many epochs, but also print the validation set accuracy at every epoch.

In [ ]:
def train_model(model, train, val, num_epochs):
    model.fit(train['data'], train['labels'], epochs=num_epochs, verbose=1, batch_size=16)
    score = model.evaluate(val['data'], val['labels'], batch_size=16, verbose=0)
    print("Validation accuracy: %.2f\n" % (score[1]))
        
train_model(model, train, val, 25)

Try playing around with hyperparameters like the learning rate, size of the hidden layers, number of epochs, etc. until you get a model that you are satisfied with! Use validation accuracy to compare performance across different model configurations. Once you're done configuring, try testing on a completely unseen dataset to get a good idea of how your model will perform for unseen data:

In [ ]:
test_score = model.evaluate(test['data'], test['labels'], batch_size=128)
print("Test accuracy: %.2f" % (test_score[1]))